The goal of this worksheet is to merfe updated DA file with the main data file

In [2]:
import pandas as pd
import numpy as np

In [3]:
da = pd.read_csv('updated_DA.csv')

In [4]:
da.head()

,Unnamed: 0,da_last_name,start year,left office,female,black,county,1976,1977,1978,...,2005-contested,2007-contested,2008-contested,2009-contested,2010-contested,2011-contested,2012-contested,2013-contested,2014-contested,2015-contested
0,0,Soares,2004,NaN,0,1,Albany,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Clyne,2000,2004.0,0,0,Albany,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Greenberg,1974,2000.0,0,0,Albany,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,Slep,2011,NaN,0,0,Allegany,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Parker,1997,2010.0,0,0,Allegany,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [5]:
main = pd.read_csv('../Raw Data/raw_criminal.csv')

In [6]:
main.head()

,Unnamed: 0,File,Casenumber,CaseName,CivilKriminal,Court,County,Judge,DistrictAttorney,ADA,...,ModeOfConviction,Crimes,Judges,Defense,DefendantAppellant,DefendantRespondent,HarmlessError,ProsecutMisconduct,DocumentLength,Justice
0,1,2009_08295.htm.txt,[67 AD3d 1368],NaN,K,County Court,Niagara County,"Peter L. Broderick, Sr., J.",Michael J. Violante,Thomas H. Brandt,...,NaN,NaN,NaN,NaN,1.0,0,NaN,NaN,857.0,NaN
1,2,2009_08296.htm.txt,[67 AD3d 1369],NaN,K,County Court,Oneida County,"Michael L. Dwyer, J.",Scott D. McNamara,Steven G. Cox,...,jury verdict,NaN,NaN,NaN,1.0,0,NaN,prosecutorial misconduct,4755.0,interest of justice
2,3,2009_08297.htm.txt,[67 AD3d 1370],NaN,K,County Court,Onondaga County,"Joseph E. Fahey, J.",William J. Fitzpatrick,Victoria M. White,...,jury verdict,NaN,NaN,Legal Aid Society,1.0,0,NaN,NaN,3951.0,interest of justice
3,4,2009_08298.htm.txt,[67 AD3d 1372],NaN,K,County Court,Niagara County,"Angelo J. Morinello, A.J.",NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,0,NaN,NaN,853.0,NaN
4,17,2009_08311.htm.txt,[67 AD3d 1378],NaN,K,County Court,Onondaga County,"Joseph E. Fahey, J.",William J. Fitzpatrick,James P. Maxwell,...,jury verdict,NaN,NaN,Legal Aid Society,1.0,0,NaN,NaN,4716.0,interest of justice


In [7]:
main.columns

Index(['Unnamed: 0', 'File', 'Casenumber', 'CaseName', 'CivilKriminal',
       'Court', 'County', 'Judge', 'DistrictAttorney', 'ADA', 'Keywords',
       'GroundsForAppeal', 'Unanimous', 'FirstDate', 'AppealDate', 'Gap_days',
       'ModeOfConviction', 'Crimes', 'Judges', 'Defense', 'DefendantAppellant',
       'DefendantRespondent', 'HarmlessError', 'ProsecutMisconduct',
       'DocumentLength', 'Justice'],
      dtype='object')

In [8]:
#Names in the main file
main_DA_names = main.DistrictAttorney.unique()
main_DA_names = [x for x in main_DA_names if str(x) != 'nan']
main_DA_names

['Michael J. Violante',
 'Scott D. McNamara',
 'William J. Fitzpatrick',
 'R. Michael Tantillo',
 'Edward M. Sharkey',
 'Donald H. Dodd',
 'Michael C. Green',
 'Lawrence Friedman',
 'Frank A. Sedita, III',
 'John H. Crandall, Sr.',
 'John C. Tunney',
 'Susan H. Lindenmuth',
 'David W. Foley',
 'Thomas E. Moran',
 'Jon E. Budelmann',
 'Robert M. Morgenthau',
 'Robert T. Johnson',
 'James A. Murphy III',
 'Nicole M. Duve',
 'P. David Soares',
 'Kevin C. Kortright',
 'Beth G. Cozzolino',
 'Gerald F. Mollen',
 'Richard D. Northrup Jr.',
 'Robert M. Carney',
 'Stephen F. Lungen',
 'Thomas J. Spota',
 'Francis D. Phillips II',
 'Thomas P. Zugibe',
 'Kathleen M. Rice',
 'Adam B. Levy',
 'Charles J. Hynes',
 'Richard A. Brown',
 'Janet DiFiore',
 'Cindy F. Intschert',
 'Kathleen B. Hogan',
 'Weeden A. Wetmore',
 'Richard J. McNally Jr.',
 'Daniel M. Donovan, Jr.',
 'Andrew J. Wylie',
 'Holley Carnright',
 'Mark D. Suben',
 'William V. Grady',
 'Richard J. McNally, Jr.',
 'Joseph A. McBride',
 

In [9]:
#Extract last name and convert to lower case

def extract_last_name(names):
    '''
    Extract lower case last name from a full name
    '''
    judges_last_names = {}
    judges = []
    
    for name in names:
        name = name.replace(', Special','')
        name = name.replace('[*2]','' )
        name = name.replace('—[*1]','' )
        name = name.strip() #remove trailing spaces
        separated = name.split(' ')
        if len(separated)>2:
            last_name = separated[2]
        elif len(separated)==2:
            last_name = separated[1]
        else:
            print(separated)
        last_name = last_name.replace(',','')
        judges_last_names[name] = last_name.lower()
        judges.append(last_name.lower())
    return judges_last_names,judges

In [10]:
judges_last_names,judges = extract_last_name(main_DA_names)

['Many']


In [11]:
judges

['violante',
 'mcnamara',
 'fitzpatrick',
 'tantillo',
 'sharkey',
 'dodd',
 'green',
 'friedman',
 'sedita',
 'crandall',
 'tunney',
 'lindenmuth',
 'foley',
 'moran',
 'budelmann',
 'morgenthau',
 'johnson',
 'murphy',
 'duve',
 'soares',
 'kortright',
 'cozzolino',
 'mollen',
 'northrup',
 'carney',
 'lungen',
 'spota',
 'phillips',
 'zugibe',
 'rice',
 'levy',
 'hynes',
 'brown',
 'difiore',
 'intschert',
 'hogan',
 'wetmore',
 'mcnally',
 'donovan',
 'wylie',
 'carnright',
 'suben',
 'grady',
 'mcnally',
 'mcbride',
 'wilhelm',
 'kortright',
 'morgenthau',
 'champagne',
 'cardone',
 'healy',
 'conboy',
 'wilkinson',
 'keene',
 'sacket',
 'gabor',
 'champagne',
 'sprague',
 'muehl',
 'northrup',
 'moser',
 'porsch',
 'stout',
 'sira',
 'vance',
 'muehl',
 'farrell',
 'rieman',
 'brown',
 'the',
 'fazzary',
 'crandall',
 'tunney',
 'tunney',
 'cook',
 'mollen',
 'rieman',
 'vance',
 'cozzolino',
 'slep',
 'vance',
 'curry',
 'vance',
 "o'geen",
 'kortwright',
 'n.y.',
 'carnright',


In [12]:
#Names in the DA file
DA_last_names = da.da_last_name.unique()

In [13]:
#Make a list of matching names for later merging
out_name = {}
not_found = []

for judge_name in judges_last_names.keys():
    found = False
    for last_name in DA_last_names:
        try:
            if last_name.lower() == judges_last_names[judge_name]:
                out_name[judge_name] = last_name
                found = True
                continue

        except (TypeError, AttributeError):
            continue
            
    if not found:
        not_found.append(judge_name)

In [14]:
out_name

{'Adam B. Levy': 'Levy',
 'Andrew J. Wylie': 'Wylie',
 'Arthur F. Glass Jr.': 'Glass',
 'Arthur Glass': 'Glass',
 'Barry L. Porsch': 'Porsch',
 'Barry Porsch': 'Porsch',
 'Beth C. Cozzolino': 'Cozzolino',
 'Beth G. Cozzolino': 'Cozzolino',
 'Brooks T. Baker': 'Baker',
 'CINDY F. INTSCHERT': 'Intschert',
 'Chad W. Brown': 'Brown',
 'Charles J. Hynes': 'Hynes',
 'Charles O. Bucca': 'Bucca',
 'Cindy F. Intschert': 'Intschert',
 'Craig P. Carriero': 'Carriero',
 'Craig Stephen Brown': 'Brown',
 'Cryus R. Vance, Jr.': 'Vance',
 'Cyrus R. Vance': 'Vance',
 'Cyrus R. Vance Jr.': 'Vance',
 'Cyrus R. Vance, Jr.': 'Vance',
 'Cyrus R. Vance, Jr., District Attorney, New York (R. Vance, Jr.': 'Vance',
 'Cyrus, R. Vance, Jr.': 'Vance',
 'D. Holley Carnright': 'Carnright',
 'D. Holley, Carnright': 'Carnright',
 'D. Holly Carnright': 'Carnright',
 'DONALD H. DODD': 'Dodd',
 'Daniel L. Master, Jr.': 'Master',
 'Daniel M. Donovan': 'Donovan',
 'Daniel M. Donovan, Jr.': 'Donovan',
 'Darcel D. Clark': 'Cl

In [15]:
#Unique names not found
set(not_found)

{'Andrew J. Bonavia',
 'Anthony A. Scarpino, Jr.',
 'Caroline A. Wojtaszek',
 'Charles E. Inman',
 'Christina Bruher',
 'Christopher Bokelman',
 'David J. Farrugia',
 'David M. Hoover',
 "Donald G. O'Geen",
 'Edward J. Nowak',
 'Eliza Filipowski',
 'Eric Gonzalez',
 'Footnote *: The',
 'George A. Farkas',
 'George M. Dentes',
 'Irene C. Graven',
 'J. Anthony Jordon',
 'James A. McCarty',
 'James P. Maxwell',
 'Jason M. Carusone',
 'John J. Flynn',
 'Jon E. Budelmann',
 'Joseph G. Fazzay',
 'Kelli P. McCoski',
 'Kevin C. Kortwright',
 'Kristyna S. Mills',
 'Lawrence Friendman',
 'Matthew Van Houten',
 'Matthew VanHouten',
 'Michael J. Flaherty',
 'Michael J. Flaherty, Jr.',
 'Michael P. Langey',
 'Mineola, N.Y.',
 'Niagara County',
 'P. David Soars',
 'Patrick A. Perfetti',
 'Patrick E. Swanson',
 'Paul L. Howard Jr.',
 'RICHARD E. SWINEHART',
 'Richard B. Stafford',
 'Richard E. Swinehart',
 'Robert Knapp',
 'Robert M. Morgenthal',
 'Robert Tendy',
 'Roger Bennet Adler',
 'Scott D. Mc 

In [16]:
#Find similar names from judges file
from difflib import get_close_matches, SequenceMatcher

lower_case_da = list(map(lambda x: x.lower(), DA_last_names))
missing = []

for name in not_found:
    judge_last_name = judges_last_names[name]
    try:
        best_match = get_close_matches(judge_last_name, lower_case_da)[0]
        score = SequenceMatcher(None, judge_last_name, best_match).ratio()
        
        if score > 0.82: #if match is close enough
            out_name[name] = last_name
            print(name, best_match, score)
        
        else:
            missing.append(name)
            print('missing:', name, judge_last_name)
               
    except IndexError:
        missing.append(name)

Jon E. Budelmann budelman 0.9411764705882353
Donald G. O'Geen o'green 0.9230769230769231
Kevin C. Kortwright kortright 0.9473684210526315
missing: Michael P. Langey langey
Richard E. Swinehart swineheart 0.9473684210526315
missing: Edward J. Nowak nowak
missing: James P. Maxwell maxwell
missing: Charles E. Inman inman
George M. Dentes dentis 0.8333333333333334
RICHARD E. SWINEHART swineheart 0.9473684210526315
Joseph G. Fazzay fazzary 0.9230769230769231
Robert M. Morgenthal morgenthau 0.9
missing: Paul L. Howard Jr. howard
P. David Soars soares 0.9090909090909091
missing: Irene C. Graven graven
Lawrence Friendman friedman 0.9411764705882353
J. Anthony Jordon jordan 0.8333333333333334
David M. Hoover hoovler 0.9230769230769231
missing: James A. McCarty mccarty
missing: Kristyna S. Mills mills
missing: Michael J. Flaherty, Jr. flaherty
missing: Niagara County county
missing: Michael J. Flaherty flaherty
missing: Roger Bennet Adler adler
missing: Patrick E. Swanson swanson
missing: Anthon

In [17]:
#Missing DA names
missing

['Footnote *: The',
 'Mineola, N.Y.',
 'Michael P. Langey',
 'Christina Bruher',
 'Edward J. Nowak',
 'James P. Maxwell',
 'Charles E. Inman',
 'George A. Farkas',
 'Scott D. Mc Namara',
 'Richard B. Stafford',
 'Paul L. Howard Jr.',
 'Irene C. Graven',
 'James A. McCarty',
 'Robert Tendy',
 'Steven R. Lewis',
 'Kristyna S. Mills',
 'Michael J. Flaherty, Jr.',
 'David J. Farrugia',
 'Niagara County',
 'Michael J. Flaherty',
 'Roger Bennet Adler',
 'Patrick E. Swanson',
 'Eric Gonzalez',
 'Andrew J. Bonavia',
 'Robert Knapp',
 'Anthony A. Scarpino, Jr.',
 'Kelli P. McCoski',
 'Theodore A. Brenner, Deputy',
 'John J. Flynn',
 'Patrick A. Perfetti',
 'Caroline A. Wojtaszek',
 'Matthew Van Houten',
 'Stephen P. Scaring',
 'Matthew VanHouten',
 'Eliza Filipowski',
 'Christopher Bokelman',
 'Stephen R. Lewis',
 'Jason M. Carusone']

#### I will match missing names by county and by year

In [18]:
missing_da_counties = main[main['DistrictAttorney'].isin(missing)].County

In [19]:
updated_main = pd.read_csv('criminal_main.csv')

In [20]:
updated_main.head()

,Unnamed: 0,File,CaseName,CivilKriminal,Court,County,Judge,DistrictAttorney,ADA,Keywords,...,Defense,First_Date,Appeal_Date,FirstD_dow,AppealD_dow,FirstD_weekday,AppealD_weekday,FirstD_month,AppealD_month,Y
0,0,2009_08295.htm.txt,NaN,K,County Court,Niagara County,"Peter L. Broderick, Sr., J.",Michael J. Violante,Thomas H. Brandt,affirmed,...,NaN,2007-11-09,2009-11-13,Friday,Friday,1,1,11.0,11,0
1,1,2009_08296.htm.txt,NaN,K,County Court,Oneida County,"Michael L. Dwyer, J.",Scott D. McNamara,Steven G. Cox,affirmed;interest of justice,...,NaN,2006-11-06,2009-11-13,Monday,Friday,1,1,11.0,11,0
2,2,2009_08297.htm.txt,NaN,K,County Court,Onondaga County,"Joseph E. Fahey, J.",William J. Fitzpatrick,Victoria M. White,affirmed;modified;interest of justice,...,Legal Aid Society,2006-10-18,2009-11-13,Wednesday,Friday,1,1,10.0,11,1
3,3,2009_08298.htm.txt,NaN,K,County Court,Niagara County,"Angelo J. Morinello, A.J.",NaN,NaN,affirmed,...,NaN,2007-11-02,2009-11-13,Friday,Friday,1,1,11.0,11,0
4,4,2009_08311.htm.txt,NaN,K,County Court,Onondaga County,"Joseph E. Fahey, J.",William J. Fitzpatrick,James P. Maxwell,affirmed;dismissed;interest of justice,...,Legal Aid Society,2005-04-21,2009-11-13,Thursday,Friday,1,1,4.0,11,1


In [21]:
updated_main.columns

Index(['Unnamed: 0', 'File', 'CaseName', 'CivilKriminal', 'Court', 'County',
       'Judge', 'DistrictAttorney', 'ADA', 'Keywords', 'GroundsForAppeal',
       'Unanimous', 'FirstDate', 'AppealDate', 'Gap_days', 'ModeOfConviction',
       'Crimes', 'Judges', 'Defense', 'First_Date', 'Appeal_Date',
       'FirstD_dow', 'AppealD_dow', 'FirstD_weekday', 'AppealD_weekday',
       'FirstD_month', 'AppealD_month', 'Y'],
      dtype='object')

In [22]:
temp = pd.DataFrame([updated_main['File'], updated_main['Appeal_Date']]).transpose()
#temp

In [23]:
#Add year column to main dataframe
main = pd.merge(main,temp, how='left', on='File')

In [24]:
def string_to_year(s):
    try:
        return int(s[0:4])
    except TypeError:
        return ''

main['appeal_year'] = list(main.Appeal_Date.map(lambda x: string_to_year(x)))

In [25]:
missing_da_years = main[main['DistrictAttorney'].isin(missing)].appeal_year

In [26]:
missing[0]

'Footnote *: The'

In [27]:
main[main.DistrictAttorney == missing[1]]

,Unnamed: 0,File,Casenumber,CaseName,CivilKriminal,Court,County,Judge,DistrictAttorney,ADA,...,Judges,Defense,DefendantAppellant,DefendantRespondent,HarmlessError,ProsecutMisconduct,DocumentLength,Justice,Appeal_Date,appeal_year
4652,6812,2011_05522.htm.txt,[85 AD3d 1055],NaN,K,Supreme Court,Nassau County,"Carter, J., at trial; Palmieri, J., at sentencing","Mineola, N.Y.",Judith R. Sternberg and Barbara Kornblau,...,"Further, the defendant argues that the verdict...",NaN,0.0,0,NaN,NaN,4638.0,NaN,2011-06-21,2011


In [28]:
main.columns

Index(['Unnamed: 0', 'File', 'Casenumber', 'CaseName', 'CivilKriminal',
       'Court', 'County', 'Judge', 'DistrictAttorney', 'ADA', 'Keywords',
       'GroundsForAppeal', 'Unanimous', 'FirstDate', 'AppealDate', 'Gap_days',
       'ModeOfConviction', 'Crimes', 'Judges', 'Defense', 'DefendantAppellant',
       'DefendantRespondent', 'HarmlessError', 'ProsecutMisconduct',
       'DocumentLength', 'Justice', 'Appeal_Date', 'appeal_year'],
      dtype='object')

In [29]:
#Add existing DA's to main frame
main['DA_last_name'] = main.DistrictAttorney.map(lambda x: out_name.get(x,'Unknown'))

In [30]:
main.head()

,Unnamed: 0,File,Casenumber,CaseName,CivilKriminal,Court,County,Judge,DistrictAttorney,ADA,...,Defense,DefendantAppellant,DefendantRespondent,HarmlessError,ProsecutMisconduct,DocumentLength,Justice,Appeal_Date,appeal_year,DA_last_name
0,1,2009_08295.htm.txt,[67 AD3d 1368],NaN,K,County Court,Niagara County,"Peter L. Broderick, Sr., J.",Michael J. Violante,Thomas H. Brandt,...,NaN,1.0,0,NaN,NaN,857.0,NaN,2009-11-13,2009,Violante
1,2,2009_08296.htm.txt,[67 AD3d 1369],NaN,K,County Court,Oneida County,"Michael L. Dwyer, J.",Scott D. McNamara,Steven G. Cox,...,NaN,1.0,0,NaN,prosecutorial misconduct,4755.0,interest of justice,2009-11-13,2009,McNamara
2,3,2009_08297.htm.txt,[67 AD3d 1370],NaN,K,County Court,Onondaga County,"Joseph E. Fahey, J.",William J. Fitzpatrick,Victoria M. White,...,Legal Aid Society,1.0,0,NaN,NaN,3951.0,interest of justice,2009-11-13,2009,Fitzpatrick
3,4,2009_08298.htm.txt,[67 AD3d 1372],NaN,K,County Court,Niagara County,"Angelo J. Morinello, A.J.",NaN,NaN,...,NaN,1.0,0,NaN,NaN,853.0,NaN,2009-11-13,2009,Unknown
4,17,2009_08311.htm.txt,[67 AD3d 1378],NaN,K,County Court,Onondaga County,"Joseph E. Fahey, J.",William J. Fitzpatrick,James P. Maxwell,...,Legal Aid Society,1.0,0,NaN,NaN,4716.0,interest of justice,2009-11-13,2009,Fitzpatrick


In [31]:

names = main[(main['appeal_year']==2012) & (main['County']=='Cayuga')].DistrictAttorney

In [43]:
index_d = {}

for name in missing:
    subset = main[main.DistrictAttorney == name]
 
    if isinstance(subset,pd.DataFrame):
        for ind, subname in subset.iterrows():
            year = subname.appeal_year
            county = subname.County
            names = main[(main['appeal_year']==year) & (main['County']==county)].DA_last_name
            print(names)
            current_name = subname.DistrictAttorney
            #names = names.replace(current_name,None)
            names = names.mode()
            val = names.tolist()
            index_d[current_name] = val
        
    else:
        year = subset.appeal_year
        county = subset.County
        names = main[(main['appeal_year']==year) & (main['County']==county)].DA_last_name
        index_d[name] = names     
            

493      Carney
573      Carney
585      Carney
590     Unknown
669      Carney
746      Carney
872     Unknown
877      Carney
922      Carney
1053    Unknown
1253     Carney
1335     Carney
1378     Carney
1534     Carney
1650    Unknown
1695     Carney
1792    Unknown
1794    Unknown
1799     Carney
2008     Carney
2015     Carney
2115     Carney
2117     Carney
2142     Carney
2174     Carney
2177     Carney
2184    Unknown
2199     Carney
2469     Carney
2573     Carney
2630     Carney
2635     Carney
2800     Carney
2801     Carney
2802     Carney
2871     Carney
2873    Unknown
2958     Carney
2968     Carney
3044     Carney
Name: DA_last_name, dtype: object
17508    Unknown
17680    Unknown
21322    Unknown
21353    Unknown
21810    Unknown
21813    Unknown
21861    Unknown
22080    Unknown
22120    Unknown
22200    Unknown
22348    Unknown
22653    Unknown
22776    Unknown
22791    Unknown
22838    Unknown
22840    Unknown
23024    Unknown
23025    Unknown
23234    Unknown
Nam

905      DiFiore
10691    DiFiore
15020    Unknown
15161    Unknown
19488    DiFiore
23364    DiFiore
23768    DiFiore
23963    DiFiore
25331    DiFiore
26683    DiFiore
26866    DiFiore
27096    DiFiore
27455    DiFiore
27818    DiFiore
28458    DiFiore
29427    DiFiore
29605    DiFiore
29760    DiFiore
30021    DiFiore
30536    DiFiore
31729    DiFiore
31912    DiFiore
32142    DiFiore
32501    DiFiore
33213    DiFiore
33492    DiFiore
34015    DiFiore
34106    DiFiore
34594    DiFiore
34666    Unknown
34669    Unknown
36589    Unknown
37619    Unknown
38858    Unknown
Name: DA_last_name, dtype: object
905      DiFiore
10691    DiFiore
15020    Unknown
15161    Unknown
19488    DiFiore
23364    DiFiore
23768    DiFiore
23963    DiFiore
25331    DiFiore
26683    DiFiore
26866    DiFiore
27096    DiFiore
27455    DiFiore
27818    DiFiore
28458    DiFiore
29427    DiFiore
29605    DiFiore
29760    DiFiore
30021    DiFiore
30536    DiFiore
31729    DiFiore
31912    DiFiore
32142    DiFio

34555    Unknown
34599    DiFiore
34610    DiFiore
34670    Unknown
34715    Unknown
34806    Unknown
34812    Unknown
34871    Unknown
34876    Unknown
34879    Unknown
34948    Unknown
34949    Unknown
34953    Unknown
34994    Unknown
35036    Unknown
35039    Unknown
35105    Unknown
35110    Unknown
35218      Vance
35225    Unknown
35312    Unknown
35321    Unknown
35368    Unknown
35375    Unknown
35491    Unknown
35497    Unknown
35503    Unknown
35505    Unknown
35556    Unknown
35565    Unknown
          ...   
36070    Unknown
36088    Unknown
36092    Unknown
36108    Unknown
36117    Unknown
36118    Unknown
36131    Unknown
36137    Unknown
36139    Unknown
36158    Unknown
36423    Unknown
36475    Unknown
36518    Unknown
36562    Unknown
36564    Unknown
36567    Unknown
36575    Unknown
36582    Unknown
36616    Unknown
36717    Unknown
36721    Unknown
36724    Unknown
36732    Unknown
36797    Unknown
36807    Unknown
36906    Unknown
36912    Unknown
36934    Unkno

34555    Unknown
34599    DiFiore
34610    DiFiore
34670    Unknown
34715    Unknown
34806    Unknown
34812    Unknown
34871    Unknown
34876    Unknown
34879    Unknown
34948    Unknown
34949    Unknown
34953    Unknown
34994    Unknown
35036    Unknown
35039    Unknown
35105    Unknown
35110    Unknown
35218      Vance
35225    Unknown
35312    Unknown
35321    Unknown
35368    Unknown
35375    Unknown
35491    Unknown
35497    Unknown
35503    Unknown
35505    Unknown
35556    Unknown
35565    Unknown
          ...   
36070    Unknown
36088    Unknown
36092    Unknown
36108    Unknown
36117    Unknown
36118    Unknown
36131    Unknown
36137    Unknown
36139    Unknown
36158    Unknown
36423    Unknown
36475    Unknown
36518    Unknown
36562    Unknown
36564    Unknown
36567    Unknown
36575    Unknown
36582    Unknown
36616    Unknown
36717    Unknown
36721    Unknown
36724    Unknown
36732    Unknown
36797    Unknown
36807    Unknown
36906    Unknown
36912    Unknown
36934    Unkno

34555    Unknown
34599    DiFiore
34610    DiFiore
34670    Unknown
34715    Unknown
34806    Unknown
34812    Unknown
34871    Unknown
34876    Unknown
34879    Unknown
34948    Unknown
34949    Unknown
34953    Unknown
34994    Unknown
35036    Unknown
35039    Unknown
35105    Unknown
35110    Unknown
35218      Vance
35225    Unknown
35312    Unknown
35321    Unknown
35368    Unknown
35375    Unknown
35491    Unknown
35497    Unknown
35503    Unknown
35505    Unknown
35556    Unknown
35565    Unknown
          ...   
36070    Unknown
36088    Unknown
36092    Unknown
36108    Unknown
36117    Unknown
36118    Unknown
36131    Unknown
36137    Unknown
36139    Unknown
36158    Unknown
36423    Unknown
36475    Unknown
36518    Unknown
36562    Unknown
36564    Unknown
36567    Unknown
36575    Unknown
36582    Unknown
36616    Unknown
36717    Unknown
36721    Unknown
36724    Unknown
36732    Unknown
36797    Unknown
36807    Unknown
36906    Unknown
36912    Unknown
36934    Unkno

...

37157    Unknown
37183    Unknown
37186    Unknown
37190    Unknown
37194    Unknown
37195    Unknown
37246    Unknown
37248    Unknown
37506    Unknown
37550    Unknown
37551    Unknown
37552    Unknown
37554    Unknown
37557    Unknown
37563    Unknown
37677    Unknown
37681    Unknown
37683    Unknown
37804    Unknown
37847    Unknown
37850    Unknown
37882    Unknown
37891    Unknown
37893    Unknown
37894    Unknown
37900    Unknown
37901    Unknown
37906    Unknown
37956    Unknown
37960    Unknown
37963    Unknown
38005    Unknown
38111    Unknown
38182    Unknown
38225    Unknown
38253    Unknown
38264    Unknown
38300    Unknown
38301    Unknown
38308    Unknown
38476    Unknown
38489    Unknown
38491    Unknown
38654    Unknown
38655    Unknown
38711    Unknown
38712    Unknown
38736    Unknown
38766    Unknown
38808    Unknown
38815    Unknown
38824    Unknown
38849    Unknown
38879    Unknown
38888    Unknown
Name: DA_last_name, dtype: object
37157    Unknown
37183    Unkno

37157    Unknown
37183    Unknown
37186    Unknown
37190    Unknown
37194    Unknown
37195    Unknown
37246    Unknown
37248    Unknown
37506    Unknown
37550    Unknown
37551    Unknown
37552    Unknown
37554    Unknown
37557    Unknown
37563    Unknown
37677    Unknown
37681    Unknown
37683    Unknown
37804    Unknown
37847    Unknown
37850    Unknown
37882    Unknown
37891    Unknown
37893    Unknown
37894    Unknown
37900    Unknown
37901    Unknown
37906    Unknown
37956    Unknown
37960    Unknown
37963    Unknown
38005    Unknown
38111    Unknown
38182    Unknown
38225    Unknown
38253    Unknown
38264    Unknown
38300    Unknown
38301    Unknown
38308    Unknown
38476    Unknown
38489    Unknown
38491    Unknown
38654    Unknown
38655    Unknown
38711    Unknown
38712    Unknown
38736    Unknown
38766    Unknown
38808    Unknown
38815    Unknown
38824    Unknown
38849    Unknown
38879    Unknown
38888    Unknown
Name: DA_last_name, dtype: object
37157    Unknown
37183    Unkno

37157    Unknown
37183    Unknown
37186    Unknown
37190    Unknown
37194    Unknown
37195    Unknown
37246    Unknown
37248    Unknown
37506    Unknown
37550    Unknown
37551    Unknown
37552    Unknown
37554    Unknown
37557    Unknown
37563    Unknown
37677    Unknown
37681    Unknown
37683    Unknown
37804    Unknown
37847    Unknown
37850    Unknown
37882    Unknown
37891    Unknown
37893    Unknown
37894    Unknown
37900    Unknown
37901    Unknown
37906    Unknown
37956    Unknown
37960    Unknown
37963    Unknown
38005    Unknown
38111    Unknown
38182    Unknown
38225    Unknown
38253    Unknown
38264    Unknown
38300    Unknown
38301    Unknown
38308    Unknown
38476    Unknown
38489    Unknown
38491    Unknown
38654    Unknown
38655    Unknown
38711    Unknown
38712    Unknown
38736    Unknown
38766    Unknown
38808    Unknown
38815    Unknown
38824    Unknown
38849    Unknown
38879    Unknown
38888    Unknown
Name: DA_last_name, dtype: object
37167    Unknown
37526    Unkno

37308    Unknown
37309    Unknown
37310    Unknown
37324    Unknown
37334    Unknown
37335    Unknown
37339    Unknown
37343    Unknown
37344    Unknown
37345    Unknown
37346    Unknown
37352    Unknown
37420    Unknown
37423    Unknown
37426    Unknown
37430    Unknown
37438    Unknown
37441    Unknown
37446    Unknown
37449    Unknown
37702    Unknown
37705    Unknown
37712    Unknown
37713    Unknown
37714    Unknown
37718    Unknown
37726    Unknown
37737    Unknown
37739    Unknown
37741    Unknown
37757    Unknown
37821    Unknown
37825    Unknown
37826    Unknown
37834    Unknown
38022    Unknown
38028    Unknown
38029    Unknown
38042    Unknown
38051    Unknown
38052    Unknown
38062    Unknown
38067    Unknown
38069    Unknown
38075    Unknown
38076    Unknown
38152    Unknown
38155    Unknown
38385    Unknown
38395    Unknown
38421    Unknown
38422    Unknown
38444    Unknown
38530    Unknown
38532    Unknown
38617    Unknown
38619    Unknown
38630    Unknown
38641    Unkno

37308    Unknown
37309    Unknown
37310    Unknown
37324    Unknown
37334    Unknown
37335    Unknown
37339    Unknown
37343    Unknown
37344    Unknown
37345    Unknown
37346    Unknown
37352    Unknown
37420    Unknown
37423    Unknown
37426    Unknown
37430    Unknown
37438    Unknown
37441    Unknown
37446    Unknown
37449    Unknown
37702    Unknown
37705    Unknown
37712    Unknown
37713    Unknown
37714    Unknown
37718    Unknown
37726    Unknown
37737    Unknown
37739    Unknown
37741    Unknown
37757    Unknown
37821    Unknown
37825    Unknown
37826    Unknown
37834    Unknown
38022    Unknown
38028    Unknown
38029    Unknown
38042    Unknown
38051    Unknown
38052    Unknown
38062    Unknown
38067    Unknown
38069    Unknown
38075    Unknown
38076    Unknown
38152    Unknown
38155    Unknown
38385    Unknown
38395    Unknown
38421    Unknown
38422    Unknown
38444    Unknown
38530    Unknown
38532    Unknown
38617    Unknown
38619    Unknown
38630    Unknown
38641    Unkno

In [44]:
index_d

{'Andrew J. Bonavia': ['Wilkinson'],
 'Anthony A. Scarpino, Jr.': ['Unknown'],
 'Caroline A. Wojtaszek': ['Unknown'],
 'Charles E. Inman': ['Cozzolino'],
 'Christina Bruher': ['Trice'],
 'Christopher Bokelman': ['Healy'],
 'David J. Farrugia': ['Unknown'],
 'Edward J. Nowak': ['Green'],
 'Eliza Filipowski': ['Unknown'],
 'Eric Gonzalez': ['Unknown'],
 'Footnote *: The': ['Unknown'],
 'Irene C. Graven': ['Unknown'],
 'James A. McCarty': ['Unknown'],
 'James P. Maxwell': ['Hartnett'],
 'Jason M. Carusone': ['Hogan'],
 'John J. Flynn': ['Unknown'],
 'Kelli P. McCoski': ['Unknown'],
 'Kristyna S. Mills': ['Unknown'],
 'Matthew Van Houten': ['Unknown'],
 'Matthew VanHouten': ['Unknown'],
 'Michael J. Flaherty': ['Unknown'],
 'Michael J. Flaherty, Jr.': ['Unknown'],
 'Michael P. Langey': ['Garcia'],
 'Mineola, N.Y.': ['Rice'],
 'Niagara County': ['Unknown'],
 'Patrick A. Perfetti': ['Unknown'],
 'Patrick E. Swanson': ['Unknown'],
 'Paul L. Howard Jr.': ['MacNeill'],
 'Robert Knapp': ['Carnri

In [34]:
final_missing = []
for key in index_d.keys():
    val = index_d[key]
    if val[0]=='Unknown':
        final_missing.append(key)
    else:
        out_name[key]=val

In [35]:
print(len(missing))

38


In [36]:
#Final people I need to check manually
final_missing

['Footnote *: The',
 'Irene C. Graven',
 'James A. McCarty',
 'Robert Tendy',
 'Kristyna S. Mills',
 'Michael J. Flaherty, Jr.',
 'David J. Farrugia',
 'Niagara County',
 'Michael J. Flaherty',
 'Patrick E. Swanson',
 'Eric Gonzalez',
 'Anthony A. Scarpino, Jr.',
 'Kelli P. McCoski',
 'Theodore A. Brenner, Deputy',
 'John J. Flynn',
 'Patrick A. Perfetti',
 'Caroline A. Wojtaszek',
 'Matthew Van Houten',
 'Matthew VanHouten',
 'Eliza Filipowski']

In [37]:
#Looks like a number of DA's are missing from the file
out_name['Irene C. Graven'] = 'Graven'
out_name['James A. McCarty'] = 'McCarty'
out_name['Robert Tendy'] = 'Tendy'
out_name['Kristyna S. Mills'] = 'Mills'
out_name['Michael J. Flaherty, Jr.'] = 'Michael J. Flaherty'
out_name['Patrick E. Swanson'] = 'Swanson'
out_name['Eric Gonzalez'] = 'Gonzalez'
out_name['Anthony A. Scarpino, Jr.'] = 'Scarpino'
out_name['Kelli P. McCoski'] = 'McCoski'
out_name['Theodore A. Brenner, Deputy'] = 'Brenner'
out_name['Theodore A. Brenner, Deputy'] = 'Brenner'
out_name['John J. Flynn'] = 'Flynn'
out_name['Patrick A. Perfetti'] = 'Perfetti'
out_name['Caroline A. Wojtaszek'] = 'Wojtaszek'
out_name['Matthew Van Houten'] = 'Van Houten'
out_name['Matthew VanHouten'] = 'Van Houten'
out_name['Eliza Filipowski'] = 'Filipowski'

In [38]:
out_name

{'Adam B. Levy': 'Levy',
 'Andrew J. Bonavia': ['Wilkinson'],
 'Andrew J. Wylie': 'Wylie',
 'Anthony A. Scarpino, Jr.': 'Scarpino',
 'Arthur F. Glass Jr.': 'Glass',
 'Arthur Glass': 'Glass',
 'Barry L. Porsch': 'Porsch',
 'Barry Porsch': 'Porsch',
 'Beth C. Cozzolino': 'Cozzolino',
 'Beth G. Cozzolino': 'Cozzolino',
 'Brooks T. Baker': 'Baker',
 'CINDY F. INTSCHERT': 'Intschert',
 'Caroline A. Wojtaszek': 'Wojtaszek',
 'Chad W. Brown': 'Brown',
 'Charles E. Inman': ['Cozzolino'],
 'Charles J. Hynes': 'Hynes',
 'Charles O. Bucca': 'Bucca',
 'Christina Bruher': ['Trice'],
 'Christopher Bokelman': ['Healy'],
 'Cindy F. Intschert': 'Intschert',
 'Craig P. Carriero': 'Carriero',
 'Craig Stephen Brown': 'Brown',
 'Cryus R. Vance, Jr.': 'Vance',
 'Cyrus R. Vance': 'Vance',
 'Cyrus R. Vance Jr.': 'Vance',
 'Cyrus R. Vance, Jr.': 'Vance',
 'Cyrus R. Vance, Jr., District Attorney, New York (R. Vance, Jr.': 'Vance',
 'Cyrus, R. Vance, Jr.': 'Vance',
 'D. Holley Carnright': 'Carnright',
 'D. Holle

In [45]:
import csv
#Write key table to csv for matching files later
with open('da_last_name_dictionary.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in out_name.items():
        if isinstance(val, list):
            val = val[0]
        writer.writerow([key, value])